### PUNTO 2.a, esto después lo cambiamos

In [4]:
from scapy.all import *

#Agrego
from scapy.layers.inet import IP, ICMP
import time
##

def ping(host):
    print (f"Ping {host}")
    
    #1. Creamos el paquete ICMP
    pkt = IP(dst=host)/ICMP()
    
    # Medimos el tiempo de envío
    start_time = time.time()
    
    # Enviamos y recibimos el paquete (timeout de 2 segundos)
    resp = sr1(pkt, timeout=2, verbose=0)
    end_time = time.time()
    
    if resp is not None and ICMP in resp and resp[ICMP].type == 0:
        # RTT (Round Trip Time)
        rtt = (end_time - start_time) * 1000  # en milisegundos
        
        # Longitud del paquete
        length = len(resp)
    
        # TTL (Time to Live)
        ttl = resp.ttl
        
        print(f"Respuesta de {host}: longitud = {length} bytes | RTT = {rtt:.2f} ms | TTL = {ttl}")
    else:
        print(f"No se recibió respuesta de {host}")

##

ping("google.com")

Ping google.com
Respuesta de google.com: longitud = 42 bytes | RTT = 71.78 ms | TTL = 118


### PUNTO 2b

In [5]:
import statistics

def ping_estadisticas(host, cantidad):
    print(f"Enviando {cantidad} paquetes ICMP a {host}...\n")
    
    enviados = 0
    recibidos = 0
    rtts = []

    for i in range(cantidad):
        pkt = IP(dst=host)/ICMP(id=0x1234, seq=i)
        enviados += 1

        start_time = time.time()
        resp = sr1(pkt, timeout=2, verbose=0)
        end_time = time.time()

        if resp is not None and ICMP in resp and resp[ICMP].type == 0:
            rtt = (end_time - start_time) * 1000  # RTT en ms
            rtts.append(rtt)
            recibidos += 1
            length = len(resp)
            ttl = resp.ttl
            print(f"[#{i+1}] Respuesta de {resp.src} | Longitud: {length} bytes | RTT: {rtt:.2f} ms | TTL: {ttl}")
        else:
            print(f"[#{i+1}] No se recibió respuesta válida")

    # Estadísticas
    perdidos = enviados - recibidos
    porcentaje_perdidos = (perdidos / enviados) * 100 if enviados > 0 else 0

    print("\nEstadísticas finales:")
    print(f"Paquetes enviados: {enviados}")
    print(f"Paquetes recibidos: {recibidos}")
    print(f"Paquetes perdidos: {perdidos}")
    print(f"Porcentaje de pérdida: {porcentaje_perdidos:.2f}%")

    if rtts:
        print(f"RTT promedio: {statistics.mean(rtts):.2f} ms")
        print(f" RTT máximo: {max(rtts):.2f} ms")
        print(f" RTT mínimo: {min(rtts):.2f} ms")
        if len(rtts) > 1:
            print(f"Desvío estándar del RTT: {statistics.stdev(rtts):.2f} ms")
        else:
            print("Desvío estándar del RTT: No se puede calcular con un solo valor")
    else:
        print("No se puede calcular RTTs: no hubo respuestas válidas")


###INPUT DE PAQUETES
host_input = input("Ingresá el host al que querés hacer ping (ej: google.com): ")
cantidad_input = int(input("¿Cuántos paquetes querés enviar?: "))

ping_estadisticas(host_input, cantidad_input)




Enviando 20 paquetes ICMP a utdt.edu...

[#1] No se recibió respuesta válida
[#2] No se recibió respuesta válida
[#3] No se recibió respuesta válida
[#4] No se recibió respuesta válida
[#5] No se recibió respuesta válida
[#6] No se recibió respuesta válida
[#7] No se recibió respuesta válida
[#8] No se recibió respuesta válida
[#9] No se recibió respuesta válida
[#10] No se recibió respuesta válida
[#11] No se recibió respuesta válida
[#12] No se recibió respuesta válida
[#13] No se recibió respuesta válida
[#14] No se recibió respuesta válida
[#15] No se recibió respuesta válida
[#16] No se recibió respuesta válida
[#17] No se recibió respuesta válida
[#18] No se recibió respuesta válida
[#19] No se recibió respuesta válida
[#20] No se recibió respuesta válida

Estadísticas finales:
Paquetes enviados: 20
Paquetes recibidos: 0
Paquetes perdidos: 20
Porcentaje de pérdida: 100.00%
No se puede calcular RTTs: no hubo respuestas válidas
